In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, gc, types
import time
from subprocess import check_output

from heapq import heappush, heappop

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/data/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_df.csv
key_1.csv
median_49.csv
page_df.csv
page_ohe.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [3]:
train = pd.read_csv(root + 'train_1.csv')

In [4]:
class Heap:
    def __init__(self, isMinHeap):
        self.isMinHeap = isMinHeap
        self.data = []
        self.location = {}

    def isInOrder(self, a, b):
        if self.isMinHeap:
            return a <= b
        else:
            return a >= b

    def add(self, id, value):
        self.data.append((id, value))
        self.location[id] = self.getSize() - 1
        currentNode = self.data[-1]
        self.siftUp(currentNode)

    def remove(self, id):
        last = self.data[-1]
        if last[0] == id:
            self.data.pop()
            return
        i = self.location[id]
        del self.location[id]
        last = self.data.pop()
        self.data[i] = last
        self.location[last[0]] = i
        self.siftUp(last)
        self.siftDown(last)

    def getRoot(self):
        if len(self.data) == 0:
            return (-1, -sys.maxint)
        return self.data[0]

    def popRoot(self):
        if len(self.data) == 0:
            return None
        first = self.data[0]
        del self.location[first[0]]
        last = self.data.pop()
        if len(self.data) > 0:
            self.data[0] = last
            self.location[last[0]] = 0
            self.siftDown(last)
        return first


    def getSize(self):
        return len(self.data)

    def siftUp(self, x):
        id = x[0]
        value = x[1]
        i = self.location[x[0]]
        while i >=1 and self.isInOrder(self.data[i][1], self.data[int((i - 1)/2)][1]):
            parent_i = int((i - 1) / 2)
            temp = self.data[parent_i]
            self.data[parent_i] = self.data[i]
            self.data[i] = temp
            self.location[self.data[i][0]] = i
            self.location[self.data[parent_i][0]] = parent_i
            i = parent_i

    def siftDown(self, x):
        id = x[0]
        value = x[1]
        i = self.location[id]
        while (2*i + 1) < self.getSize():
            child_i = 2*i + 1
            child_i_2 = 2*i + 2
            if child_i_2 < self.getSize() and self.isInOrder(self.data[child_i_2][1], self.data[child_i][1]) :
                child_i = child_i_2
            if self.isInOrder(self.data[child_i][1], self.data[i][1]):
                temp = self.data[child_i]
                self.data[child_i] = self.data[i]
                self.data[i] = temp
                self.location[self.data[i][0]] = i
                self.location[self.data[child_i][0]] = child_i
            i = child_i

In [5]:
def medianSlidingWindow(nums, k):
    # write your code here
    if nums is None or len(nums) == 0:
        return []
    n = len(nums)
    result = []
    maxHeap = Heap(False) #half smaller numbers 
    minHeap = Heap(True)


    for i in range(0, n):
        first = i - k
        if first >= 0 and first in maxHeap.location:
            maxHeap.remove(first)
        elif first >= 0 and first in minHeap.location:
            minHeap.remove(first)
        if i == 0 or nums[i] <= maxHeap.getRoot()[1]:
            maxHeap.add(i, nums[i])
        else:
            minHeap.add(i, nums[i])
        if maxHeap.getSize() - minHeap.getSize() >= 2:
            top = maxHeap.popRoot()
            minHeap.add(top[0], top[1])
        elif maxHeap.getSize() - minHeap.getSize() <= -1:
            top = minHeap.popRoot()
            maxHeap.add(top[0], top[1])
        if i >= k - 1:
            result.append(maxHeap.getRoot()[1])

    return result

In [6]:
def medianStreaming(nums):
    # write your code here
    if nums is None or len(nums) == 0:
        return None
    n = len(nums)

    result = [nums[0]]
    maxHeap = [(-nums[0], nums[0])] #half smaller numbers 
    minHeap = [] #half larger numbers

    for i in range(1, n):
        if nums[i] <= maxHeap[0][1]:
            heappush(maxHeap, (-nums[i], nums[i]))
        else:
            heappush(minHeap, (nums[i], nums[i]))
        if len(maxHeap) - len(minHeap) == 2:
            top = heappop(maxHeap)
            heappush(minHeap, (top[1], top[1]))
        elif len(maxHeap) - len(minHeap) == -1:
            top = heappop(minHeap)
            heappush(maxHeap, (-top[1], top[1])) 
        result.append(maxHeap[0][1])

    return result

In [7]:
train.fillna(0, inplace = True)
median_df = train.copy()

In [8]:
window = 49

In [9]:
train_np = train.iloc[:, 1:].values
median_np = median_df.iloc[:, 1:].values
for i in range(len(median_df)):
# for i in range(1000):
    nums = train_np[i, :]
    cur_median = medianStreaming(nums[:window - 1]) + medianSlidingWindow(nums, window)
    median_np[i, :] = cur_median
    if i % 100 == 0:
        print('row {} is done'.format(i))
median_df.iloc[:, 1:] = median_np                            

row 0 is done
row 100 is done
row 200 is done
row 300 is done
row 400 is done
row 500 is done
row 600 is done
row 700 is done
row 800 is done
row 900 is done
row 1000 is done
row 1100 is done
row 1200 is done
row 1300 is done
row 1400 is done
row 1500 is done
row 1600 is done
row 1700 is done
row 1800 is done
row 1900 is done
row 2000 is done
row 2100 is done
row 2200 is done
row 2300 is done
row 2400 is done
row 2500 is done
row 2600 is done
row 2700 is done
row 2800 is done
row 2900 is done
row 3000 is done
row 3100 is done
row 3200 is done
row 3300 is done
row 3400 is done
row 3500 is done
row 3600 is done
row 3700 is done
row 3800 is done
row 3900 is done
row 4000 is done
row 4100 is done
row 4200 is done
row 4300 is done
row 4400 is done
row 4500 is done
row 4600 is done
row 4700 is done
row 4800 is done
row 4900 is done
row 5000 is done
row 5100 is done
row 5200 is done
row 5300 is done
row 5400 is done
row 5500 is done
row 5600 is done
row 5700 is done
row 5800 is done
row 5900 

In [10]:
median_df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,2016-04-25,2016-04-26,2016-04-27,2016-04-28,2016-04-29,2016-04-30,2016-05-01,2016-05-02,2016-05-03,2016-05-04,2016-05-05,2016-05-06,2016-05-07,2016-05-08,2016-05-09,2016-05-10,2016-05-11,2016-05-12,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,2016-05-31,2016-06-01,2016-06-02,2016-06-03,2016-06-04,2016-06-05,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19,2016-06-20,2016-06-21,2016-06-22,2016-06-23,2016-06-24,2016-06-25,2016-06-26,2016-06-27,2

In [11]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,2016-04-25,2016-04-26,2016-04-27,2016-04-28,2016-04-29,2016-04-30,2016-05-01,2016-05-02,2016-05-03,2016-05-04,2016-05-05,2016-05-06,2016-05-07,2016-05-08,2016-05-09,2016-05-10,2016-05-11,2016-05-12,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,2016-05-31,2016-06-01,2016-06-02,2016-06-03,2016-06-04,2016-06-05,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19,2016-06-20,2016-06-21,2016-06-22,2016-06-23,2016-06-24,2016-06-25,2016-06-26,2016-06-27,2

In [12]:
median_df.to_csv(root+'median_49.csv', index = None)

In [4]:
train_melt = train.melt(id_vars=['Page'], var_name='date')

In [6]:
train_melt.head()

,Page,date,value
0,2NE1_zh.wikipedia.org_all-access_spider,2015-07-01,18.0
1,2PM_zh.wikipedia.org_all-access_spider,2015-07-01,11.0
2,3C_zh.wikipedia.org_all-access_spider,2015-07-01,1.0
3,4minute_zh.wikipedia.org_all-access_spider,2015-07-01,35.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,2015-07-01,NaN
